In [ ]:
import os
sys.path.append("/opt/miniconda3/lib/python3.7/site-packages") 
import tensorflow_hub as hub
import numpy as np
import pandas as pd


In [ ]:
# load a pre-trained embedding
# Token based text embedding trained on Chinese Google News 100B corpus.
# https://tfhub.dev/google/nnlm-zh-dim50/2
embed = hub.load("https://tfhub.dev/google/nnlm-zh-dim50/2")


In [ ]:
def cos_sim(vector_a, vector_b):
    """
    计算两个向量之间的余弦相似度
    :param vector_a: 向量 a 
    :param vector_b: 向量 b
    :return: sim
    """
    vector_a = np.mat(vector_a)
    vector_b = np.mat(vector_b)
    
    num = float(vector_a * vector_b.T)
    denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
    
    cos = num / denom
    sim = 0.5 + 0.5 * cos
    
    return sim

def embeddings_cos_sim(ab):
    embeddings = embed(ab)
    B=embeddings.numpy()[1]
    A=embeddings.numpy()[0]
    return cos_sim(A, B)
   # print(ab, cos_sim(A, B))

In [ ]:
personality = ['纯朴的','顾家的','小镇的','道德的','真实的','守信的','新颖的','纯正的','永恒的','感情的','友善的','温馨的','时髦的','刺激的','非凡的','酷酷的','年轻的','外向的','独特的','幽默的','文艺的','进取的','独立的','当代的','勤奋的','安全的','有效的','技术的','合群的','认真的','自信的','领导的','气场的','魅力的','好看的','卓尔的','温和的','女性的','柔顺的','阳刚的','西部的','积极的','强硬的','粗犷的','耿直的']

In [ ]:
brandData = pd.read_csv('../data/品牌信息.csv')

for i in range(1,len(brandData)):
    data = brandData.iloc[i] 
    print(data[0]+'\n')
    # brandPersonalityMin(data)
    brandPersonalityMean(data)


In [ ]:
# 找最小值
def brandPersonalityMin(data):
    dataNeed = (data[23] +' / '+ data[24] +' / '+ data[25]).split(' / ')
    m = 0
    p = 'test'
    k = 'test'
    for i in dataNeed:
        for j in personality:
            similar = embeddings_cos_sim([i,j])
            if similar > m:
                m = similar
                p = j
                k = i
    print('品牌个性：' + str(p) + '\n相似度：' + str(m) + '\n关键词：' + str(k) +'\n')


In [ ]:
# 找均值
def brandPersonalityMean(data):
    dataNeed = (data[23] +' / '+ data[24] +' / '+ data[25]).split(' / ')
    vector = 0
    for i in dataNeed:
        vector += embed([i])
    vector = vector / len(dataNeed)

    m = 0
    p = 'test'
    for i in personality:
        per = embed([i])
        similar = cos_sim(vector, per)
        if similar > m:
            m = similar
            p = i

    print('品牌个性：' + str(p) + '\n相似度：' + str(m) +'\n')